In [1]:
import os
from collections import Counter
import pandas as pd
import zipfile
from tqdm import tqdm
from glob import glob

In [2]:
ROOT_DIR = '/media/sjb/Sandra M/SJ_Partition/unzipped/'

In [3]:
all_unzipped_dirs = []

for top_dir in os.listdir(ROOT_DIR):
    top_dir_expanded = os.path.join(ROOT_DIR, top_dir)
    for sub_dir in os.listdir(top_dir_expanded):
        all_unzipped_dirs.append({
            'lowest_dir': sub_dir,
            'parent_dir': top_dir,
        })

In [4]:
len(all_unzipped_dirs)

1547

In [5]:
df = pd.DataFrame(all_unzipped_dirs)
df.shape

(1547, 2)

## 1. Check Duplicates - folder 1000 is duplicate

In [6]:
df['lowest_dir'].value_counts().head()

1000    2
525     1
360     1
482     1
128     1
Name: lowest_dir, dtype: int64

In [7]:
df.query('lowest_dir=="1000"')

,lowest_dir,parent_dir
0,1000,1000_1500
1047,1000,First_1000_1


#### (a) check 1000_1500/1000 vs. First_1000_1/1000

In [8]:
files1 = set([os.path.basename(x) for x in glob(os.path.join(ROOT_DIR, '1000_1500', '1000', '*.html'))])
files2 = set([os.path.basename(x) for x in glob(os.path.join(ROOT_DIR, 'First_1000_1', '1000', '*.html'))])

In [9]:
files1.symmetric_difference(files2)

set()

## 2. Check the directory list vs archive

In [10]:
first_1000_df = df[df['parent_dir'].str.startswith('First')].copy()

In [11]:
set1 = set(first_1000_df['lowest_dir'].astype(int))
set2 = set(range(1, 1001))

In [12]:
set1.symmetric_difference(set2)

set()

In [13]:
archive_file_root = '/media/sjb/Sandra M/'

In [14]:
archive_file_path = os.path.join(archive_file_root, '2000.zip')
archive_file_path

'/media/sjb/Sandra M/2000.zip'

In [15]:
def _is_dir(filename):
    """Heuristic"""
    return filename.endswith('/')

In [16]:
all_archive_file_dirs = set()

with zipfile.ZipFile(archive_file_path, 'r')  as archive:
    # Get a list of all archived file names from the zip
    file_names = archive.namelist()
    # Iterate over the file names
    for file_name in file_names:
        # Check filename endswith csv
        if _is_dir(file_name):
            all_archive_file_dirs.add(file_name.replace('/', ''))
        

In [21]:
set_unzipped = set(df.query('parent_dir=="2000"')['lowest_dir'])

In [22]:
len(set_unzipped)

111

In [23]:
all_archive_file_dirs.symmetric_difference(set_unzipped)

set()